# Chain using LangGraph

LangGraph uses 4 important concepts

- How to use chat messages as our graph state
- How to use chat models in graph nodes
- How to bind tools to our LLM in chat models
- How to execute the tools call in out graph nodes

In [1]:
from dotenv import load_dotenv

load_dotenv()

import os
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

# How to use chat messages a out graph state

Messages

We can use messages which can be used to capture different roles with in a conversation. LangChain has various message typed incliding HumanMessage, AIMessage, SystemMessage and ToolMessage. These represent a message from the user, from chat model, 

Every message have these important components.

- content- content of the message
- name- Specify the name of author
- response_metadata - Optionally, a dict of metadata (eg., often populated by model provider for AIMessages)


In [4]:
from langchain_core.messages import AIMessage, HumanMessage
from pprint import pprint # this lib is to print in beautiful manner (pretty print)

messages = [AIMessage(content=f"Please tellme how can I help", name="LLM-Model")]
messages.append(HumanMessage(content=f"I want to learn coding!", name="BoonSai"))
messages.append(AIMessage(content=f"Which programming languge you want to learn", name="LLM-Model"))
messages.append(HumanMessage(content=f"Hey! I want to learn python programming langugae", name="BoonSai"))

for message in messages: message.pretty_print()


================================== Ai Message ==================================
Name: LLM-Model

Please tellme how can I help
================================ Human Message =================================
Name: BoonSai

I want to learn coding!
================================== Ai Message ==================================
Name: LLM-Model

Which programming languge you want to learn
================================ Human Message =================================
Name: BoonSai

Hey! I want to learn python programming langugae


# Chat models

We can use the sequence of messages a input with the chat-models using LLM's

In [8]:
from langchain_groq import ChatGroq

llm = ChatGroq(model = 'llama-3.1-8b-instant')
result = llm.invoke(messages)

In [11]:
result.response_metadata # it prints metadata of the llm's output

{'token_usage': {'completion_tokens': 491,
  'prompt_tokens': 82,
  'total_tokens': 573,
  'completion_time': 0.688799497,
  'prompt_time': 0.004415927,
  'queue_time': 0.044415641,
  'total_time': 0.693215424},
 'model_name': 'llama-3.1-8b-instant',
 'system_fingerprint': 'fp_a7a2f9abbf',
 'service_tier': 'on_demand',
 'finish_reason': 'stop',
 'logprobs': None}

# Tools

-Tools can be integrated with the LLM models to interact with external systems. External systems can be API's, third party tools. 

- Whenever a query is asked the model can choose to call the tool and this query is based on the natural language input and this will return and output that matches the tool's schema

In [12]:
def add(a: int, b:int)-> int:
    """ Add a and b
    Args:
        a (int): First int
        b (int): second int
    
    returns:
        int     
    """
    return a+b 

In [13]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x124a01d10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x124a75c90>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [15]:
## Binding tool with llm

llm_with_tools = llm.bind_tools([add])

llm_with_tools.invoke([HumanMessage(content=f"what is 2 plus 2", name='BoonSai')]) # here it shows empty message 

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'wxbvaw164', 'function': {'arguments': '{"a":2,"b":2}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 269, 'total_tokens': 287, 'completion_time': 0.038753537, 'prompt_time': 0.042570129, 'queue_time': 0.04549145, 'total_time': 0.081323666}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_3ddc9808b3', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--c26f1654-1262-4fd8-b9f8-ad2fe5737894-0', tool_calls=[{'name': 'add', 'args': {'a': 2, 'b': 2}, 'id': 'wxbvaw164', 'type': 'tool_call'}], usage_metadata={'input_tokens': 269, 'output_tokens': 18, 'total_tokens': 287})